# OpenCV Calibration

In [2]:
try:
    import cv2 as cv
except ImportError:
    %pip install opencv-python
    import cv as cv

## Video and Image File Processing

### Video File Processing

In this section, we go through the process of creating an application that will:
- Display images from a camera or various files
- Convert a color image to a grayscale image
- Allow the user to toggle between the original and grayscale images

In [5]:
import os
ESC_KEY = 27
SPACE_KEY = 32

def video_display(video_path=None):
    """
    Displays video from a given path or webcam.
    This function captures video from a specified file path or the default webcam if no path is provided.
    It displays the video in a window, allowing the user to toggle between color and grayscale views by pressing the SPACE key.
    Pressing the ESC key will exit the video display.
    Parameters:
        video_path (str, optional): The file path to the video to be displayed. Defaults to None, which uses the webcam.
    Returns:
        None
    Raises:
        IOError: If the provided video_path does not exist or cannot be opened.
        Exception: If an unexpected error occurs during video processing.
    """

    if video_path is None:
        video_path = 0  # Use webcam
    elif not os.path.isfile(video_path):
        print(f"Error: The file '{video_path}' does not exist.")
        return

    cap = cv.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video source.")
        return 

    grayscale = False

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            # Convert the frame to grayscale if the user has toggled the display
            if grayscale:
                display_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            else:
                display_frame = frame

            cv.imshow('Video', display_frame)

            # Check for the user pressing the ESC or SPACE key
            key = cv.waitKey(1) & 0xFF # 0xFF is used to get the lowest byte of the return value
            if key == ESC_KEY:  
                break
            elif key == SPACE_KEY:  
                grayscale = not grayscale
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cap.release()
        cv.destroyAllWindows()

video_display()

### Image File Processing

Now, we are going to try to display images instead of videos. We will use the same code as before, but we will change the video capture to image capture.

These images are stored in the `calib_gopro` folder. 

In [12]:
def image_display():
    """
    Displays images from the 'calib_gopro' folder.
    Allows toggling between color and grayscale images.
    """
    import glob

    ESC_KEY = 27
    SPACE_KEY = 32
    NEXT_KEY = ord('n')
    PREV_KEY = ord('p')

    # Load images from the 'calib_gopro' folder
    image_folder = 'calib_gopro'
    image_pattern = os.path.join(image_folder, 'GOPR84*.JPG')
    image_files = sorted(glob.glob(image_pattern))

    if not image_files:
        print("No images found in the folder.")
        return

    index = 0
    grayscale = False

    while True:
        image_path = image_files[index]
        frame = cv.imread(image_path)

        if frame is None:
            print(f"Failed to load image: {image_path}")
            break

        while True:
            if grayscale:
                display_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            else:
                display_frame = frame

            cv.imshow('Image Viewer', display_frame)

            # Wait for user input to switch images or modes
            key = cv.waitKey(0) & 0xFF

            if key == ESC_KEY:
                cv.destroyAllWindows()
                return
            elif key == SPACE_KEY:
                grayscale = not grayscale
            elif key == NEXT_KEY:
                index = (index + 1) % len(image_files)
                break
            elif key == PREV_KEY:
                index = (index - 1) % len(image_files)
                break

    cv.destroyAllWindows()

image_display()

## Camera Calibration with OpenCV

Now the goal is to calibrate the camera using a chessboard pattern. Once enough images are captured, we'll be able to calculate the camera matrix and distortion coefficients. Finally, we'll try to straighten the input camera image

![image0.png](./attachments/image0.png)

### Chess board detection and camera calibration

Camera calibration involves identifying a set of distinctive points on a specific target, such as the inner corners of a chessboard, and determining the intrinsic parameters along with the position and orientation of these points.

We will use the `cv2.findChessboardCorners()` function to find the corners of the chessboard. This function returns the corners of the chessboard and a boolean value that indicates whether the corners were found or not.

Next, we will use the `cv2.drawChessboardCorners()` function to draw the corners on the image.

Finally, the retuned corners will be used to calibrate the camera using the `cv2.calibrateCamera()` function.

In order to calibrate the camera, we need to show to the camera a chessboard pattern from different angles. The more angles we show, the better the calibration will be (the number of corners can be changed according to the chessboard pattern).

In [36]:
def camera_calibration(num_corners_hor, num_corners_ver, num_frames=20):
    """
    Perform camera calibration using a chessboard pattern from a webcam stream.

    Args:
        num_corners_hor (int): Number of inner corners along the chessboard's width.
        num_corners_ver (int): Number of inner corners along the chessboard's height.
        num_frames (int): Number of valid frames to collect for calibration.

    Returns:
        ret (float): Overall RMS re-projection error.
        camera_matrix (numpy.ndarray): Camera matrix (intrinsic parameters).
        dist_coeffs (numpy.ndarray): Distortion coefficients.
        rvecs (list): Rotation vectors.
        tvecs (list): Translation vectors.
        captured_frames (list): List of captured frames used for calibration.
    """
    import numpy as np

    # Generate the object points for a single chessboard pattern
    objp = np.zeros((num_corners_ver * num_corners_hor, 3), np.float32)
    objp[:, :2] = np.mgrid[0:num_corners_hor, 0:num_corners_ver].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    object_points = []  # 3D points in real world space
    image_points = []   # 2D points in image plane
    captured_frames = []  # List to store captured frames

    # Open webcam stream
    cap = cv.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return None, None, None, None, None, None

    # Define the size of the chessboard (number of inner corners)
    board_size = (num_corners_hor, num_corners_ver)

    print(f"Press SPACE to capture frame {num_frames} valid frames. Press 'q' or SPACE KEY to quit early.")

    collected_frames = 0

    while collected_frames < num_frames:
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from webcam.")
            break

        # Convert to grayscale
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Detect chessboard corners
        found, corners = cv.findChessboardCorners(
            gray_frame,
            board_size,
        )

        if found:
            # Refine corner detection
            criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            corners2 = cv.cornerSubPix(gray_frame, corners, (11, 11), (-1, -1), criteria)

            # Draw the detected corners
            cv.drawChessboardCorners(frame, board_size, corners2, found)

        # Display the frame
        cv.imshow('Webcam Chessboard Calibration', frame)

        key = cv.waitKey(1) & 0xFF

        if key == SPACE_KEY:  # Space key to capture
            if found:
                # Store the object points and image points
                object_points.append(objp)
                image_points.append(corners2)

                # Store the captured frame
                captured_frames.append(frame.copy())

                collected_frames += 1
                print(f"Captured frame {collected_frames}/{num_frames}")
            else:
                print("Chessboard not detected. Adjust the board and try again.")

        elif key == ESC_KEY or key == ord('q'):
            print("Calibration interrupted by user.")
            break

    # Release the webcam and close all OpenCV windows
    cap.release()
    cv.destroyAllWindows()

    if collected_frames >= 1:
        image_size = gray_frame.shape[::-1]

        # Perform camera calibration
        ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv.calibrateCamera(
            object_points, image_points, image_size, None, None
        )

        return ret, camera_matrix, dist_coeffs, rvecs, tvecs, captured_frames
    else:
        print("Not enough frames collected for calibration.")
        return None, None, None, None, None, None

In [37]:
# Perform camera calibration
num_corners_hor = 9
num_corners_ver = 6
num_frames = 20
ret, camera_matrix, dist_coeffs, rvecs, tvecs, captured_frames = camera_calibration(num_corners_hor, num_corners_ver, num_frames)

Press SPACE to capture frame 20 valid frames. Press 'q' or SPACE KEY to quit early.
Captured frame 1/20
Captured frame 2/20
Captured frame 3/20
Captured frame 4/20
Captured frame 5/20
Captured frame 6/20
Captured frame 7/20
Captured frame 8/20
Captured frame 9/20
Captured frame 10/20
Captured frame 11/20
Captured frame 12/20
Captured frame 13/20
Captured frame 14/20
Captured frame 15/20
Captured frame 16/20
Captured frame 17/20
Captured frame 18/20
Chessboard not detected. Adjust the board and try again.
Chessboard not detected. Adjust the board and try again.
Captured frame 19/20
Captured frame 20/20


### Undistorting the input image

Now that we have the camera matrix and distortion coefficients, we can use the `cv2.undistort()` function to undistort the input image.

In [40]:
def undistort_captured_images(captured_frames):
    """
    Undistorts the captured images using the precomputed camera matrix and distortion coefficients.

    Args:
        captured_frames (list): List of captured frames to be undistorted.

    Returns:
        undistorted_images (list): List of undistorted images.
    """
    undistorted_images = []

    print("Press SPACE to go to the next image. Press ESC to exit.")
    for i, frame in enumerate(captured_frames):
        # Undistort the image
        undistorted_image = cv.undistort(frame, camera_matrix, dist_coeffs)
        undistorted_images.append(undistorted_image)

        while True:
            # Display the original and undistorted images
            cv.imshow(f'Original Image {i+1}', frame)
            cv.imshow(f'Undistorted Image {i+1}', undistorted_image)
            key = cv.waitKey(0) & 0xFF

            if key == ESC_KEY:
                cv.destroyAllWindows()
                return undistorted_images
            elif key == SPACE_KEY:
                cv.destroyAllWindows()
                break

    return undistorted_images

# Undistort the captured images
undistorted_images = undistort_captured_images(captured_frames)


When showing the undistorted image, we can see black areas around the image. This is because the image has been undistorted and the corners of the image have been moved. 

## Calibrate from a group of images

Now, instead of using a video stream, we will use a group of images to calibrate the camera.
The images are stored in the `calib_gopro` folder.

In [52]:
def calibrate_camera_from_folder(num_corners_hor, num_corners_ver):
    """
    Calibrates the camera using images of a chessboard pattern stored in a specified folder.

    Args:
        num_corners_hor (int): Number of inner corners along the chessboard's width.
        num_corners_ver (int): Number of inner corners along the chessboard's height.

    Returns:
        ret (float): Overall RMS re-projection error.
        camera_matrix (numpy.ndarray): Camera matrix (intrinsic parameters).
        dist_coeffs (numpy.ndarray): Distortion coefficients.
        rvecs (list): Rotation vectors.
        tvecs (list): Translation vectors.
        images_used (list): List of image file paths used for calibration.
    """
    import glob
    import numpy as np

    # Termination criteria for cornerSubPix
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points based on the real chessboard dimensions
    objp = np.zeros((num_corners_ver * num_corners_hor, 3), np.float32)
    objp[:, :2] = np.mgrid[0:num_corners_hor, 0:num_corners_ver].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    object_points = []  # 3D points in real world space
    image_points = []   # 2D points in image plane
    images_used = []     # List to store paths of images used

    # Load images from the 'calib_gopro' folder
    image_folder = 'calib_gopro'
    image_pattern = os.path.join(image_folder, 'GOPR84*.JPG')
    image_files = sorted(glob.glob(image_pattern))

    if not image_files:
        print("No images found in the specified folder.")
        return None, None, None, None, None, None

    for fname in image_files:
        img = cv.imread(fname)
        if img is None:
            print(f"Failed to load image: {fname}")
            continue

        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Find chessboard corners
        found, corners = cv.findChessboardCorners(gray, (num_corners_hor, num_corners_ver), None)

        if found:
            # Refine corner locations
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

            # Append object points and image points
            object_points.append(objp)
            image_points.append(corners2)
            images_used.append(fname)

            # Draw and display the corners
            cv.drawChessboardCorners(img, (num_corners_hor, num_corners_ver), corners2, found)
            cv.imshow('Chessboard Detection', img)
            cv.waitKey(100)  # Display each valid image for 100 ms
        else:
            print(f"Chessboard not detected in image: {fname}")

    cv.destroyAllWindows()

    if not object_points or not image_points:
        print("No valid images found for calibration.")
        return None, None, None, None, None, None

    # Get image size from the first valid image
    image_size = gray.shape[::-1]

    # Perform camera calibration
    print(f"Calibrating using {len(object_points)} images...")
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv.calibrateCamera(
        object_points, image_points, image_size, None, None
    )

    # Convert the images to numpy arrays
    frames = []
    for image in images_used:
        img = cv.imread(image)
        if img is not None:
            frames.append(img)
        else:
            print(f"Failed to load image: {image}")

    return ret, camera_matrix, dist_coeffs, rvecs, tvecs, frames

In [53]:
# Calibrate camera using images from the 'calib_gopro' folder
num_corners_hor = 9
num_corners_ver = 6
ret, camera_matrix, dist_coeffs, rvecs, tvecs, images_used = calibrate_camera_from_folder(num_corners_hor, num_corners_ver)

Chessboard not detected in image: calib_gopro\GOPR8411.JPG
Calibrating using 26 images...


Now that the images from the folder have been used to calibrate the camera, we can read the images from the folder and undistort them using the camera matrix and distortion coefficients.

In [54]:
undistorted_images = undistort_captured_images(images_used)

We can see that the images have been undistorted since the chessboard pattern is now straight.